## RFM(Recency, Frequency, Monetary)

* RFM은 가치있는 고객을 추출해내어 이를 기준으로 고객을 분류할 수 있는 매우 간단하면서도 유용하게 사용될 수 있는 방법으로 알려져 있어 마케팅에서 가장 많이 사용되고 있는 분석방법 중 하나이다. RFM은 구매 가능성이 높은 고객을 선정하기 위한 데이터 분석방법으로서, 분석과정을 통해 데이터는 의미있는 정보로 전환된다.

* RFM은 Recency, Frequency, Monetary의 약자로 고객의 가치를 다음의 세 가지 기준에 의해 계산하고 있다.

* Recency - 거래의 최근성: 고객이 얼마나 최근에 구입했는가?
* Frequency - 거래빈도: 고객이 얼마나 빈번하게 우리 상품을 구입했나?
* Monetary - 거래규모: 고객이 구입했던 총 금액은 어느 정도인가?

### RFM 모형

* Scoring 기법: RFM의 요인을 각각 5등급으로 등간격으로 분류하는 방법이다.
* 현재 개발된 RFM 모형은 크게 4가지로 분류 할 수 있다. 이 문서의 내용들은 정설이 아니며 신뢰하기 어려움을 전제로 참고해야한다.
    * 모델1. RFM 각 요소의 20% rule의 적용
    * 모델2. 비율 척도에 의한 양적인 정도의 차이에 따른 등간격의 5등급 분류
    * 모델3. 상하 20%를 제외한 등간격 척도에 의한 그룹 분류
    * 모델4. 군집 분석에 의한 각 요소 별 5개의 그룹 분류
* Data Mining 기법을 이용한 모형
* 회귀분석
* 선형 회귀 분석을 이용한 모형: 고객의 구매 최근성, 구매 빈도, 구매 금액 등 고객의 수익 기여도를 나타내는 세가지 지표들의 선형결합으로 세가지 지표들을 점수화 한다.
* 다중 회귀 분석을 이용한 모형: 각 고객의 구매 행동을 나타내는 R,F,M의 변수들을 독립변수로하고 고객의 미래 구매 행동을 예측하는 기법.
* 신경망을 적용한 모형: 로지스틱 회귀 모형을 보완하는 차원에서 연구.
* 확률적 RFM모형:Colombo와 Weina의 확률적인 RFM모형은 과거의 고객의 응답 이력으로 고객의 미래 응답을 예측하는 행동모델이다.

출처 : [RFM - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/RFM)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

In [3]:
# df = pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")
df = pd.read_excel("data/Online Retail.xlsx")
df.shape

(541909, 8)

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
df = df[df["CustomerID"].notnull() & (df["Quantity"] > 0)].copy()
df.shape

(397924, 8)

In [6]:
df["TotalPrice"] = df["UnitPrice"] * df["Quantity"]

In [7]:
# 해당 주문에서 가장 최근 구매가 일어난 시간을 가져옴
last_timestamp = df['InvoiceDate'].max() + dt.timedelta(days = 1)
last_timestamp

Timestamp('2011-12-10 12:50:00')

In [8]:
# Recency : 최근 날짜 기준으로 마지막으로 구매한 날짜와 얼마나 차이가 나는지
# Frequency : 구매 빈도수
# Monetary : 총 구매 금액
rfm = df.groupby(['CustomerID']).agg({'InvoiceDate': lambda x : (last_timestamp - x.max()).days,
                                      'InvoiceNo':'count','TotalPrice': 'sum'})

In [9]:
rfm.rename(columns={'InvoiceDate':'Recency', 
                    'InvoiceNo':'Frequency', 
                    'TotalPrice':'MonetaryValue'},
           inplace= True)
rfm

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40
...,...,...,...
18280.0,278,10,180.60
18281.0,181,7,80.82
18282.0,8,12,178.05


In [13]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,4339.000000,4339.000000,4339.000000
mean,92.518322,91.708689,2053.793018
std,100.009747,228.792852,8988.248381
min,1.000000,1.000000,0.000000
25%,18.000000,17.000000,307.245000
50%,51.000000,41.000000,674.450000
75%,142.000000,100.000000,1661.640000
max,374.000000,7847.000000,280206.020000


In [41]:
r_labels = range(5, 0, -1)
f_labels = range(1, 6)
m_labels = range(1, 6)
cut_size= 5
r_cut = pd.qcut(rfm['Recency'], q=cut_size, labels = r_labels)
f_cut = pd.qcut(rfm['Frequency'],q=cut_size, labels = f_labels)
m_cut = pd.qcut(rfm['MonetaryValue'],q=cut_size,labels = m_labels)
rfm = rfm.assign(R=r_cut, F=f_cut, M=m_cut)

In [42]:
rfm

,Recency,Frequency,MonetaryValue,R,F,M,RFM,RFM_score
CustomerID,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,NaN
12347.0,2,182,4310.00,5,5,5,555,NaN
12348.0,75,31,1797.24,2,3,4,234,NaN
12349.0,19,73,1757.55,4,4,4,444,NaN
12350.0,310,17,334.40,1,2,2,122,NaN
...,...,...,...,...,...,...,...,...
18280.0,278,10,180.60,1,1,1,111,NaN
18281.0,181,7,80.82,1,1,1,111,NaN
18282.0,8,12,178.05,5,1,1,511,NaN


In [49]:
rfm["RFM_segment"] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)

In [50]:
rfm["RFM_score"] = rfm[["R", "F", "M"]].astype(int).sum(axis=1)
rfm

,Recency,Frequency,MonetaryValue,R,F,M,RFM,RFM_score,RFM_segment
CustomerID,,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,7,115
12347.0,2,182,4310.00,5,5,5,555,15,555
12348.0,75,31,1797.24,2,3,4,234,9,234
12349.0,19,73,1757.55,4,4,4,444,12,444
12350.0,310,17,334.40,1,2,2,122,5,122
...,...,...,...,...,...,...,...,...,...
18280.0,278,10,180.60,1,1,1,111,3,111
18281.0,181,7,80.82,1,1,1,111,3,111
18282.0,8,12,178.05,5,1,1,511,7,511


In [70]:
rfm["RFM_class"] = pd.qcut(rfm["RFM_score"], 3, labels=["silver", "gold", "platinum"])
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM,RFM_score,RFM_segment,RFM_class
CustomerID,,,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,7,115,silver
12347.0,2,182,4310.00,5,5,5,555,15,555,platinum
12348.0,75,31,1797.24,2,3,4,234,9,234,gold
12349.0,19,73,1757.55,4,4,4,444,12,444,platinum
12350.0,310,17,334.40,1,2,2,122,5,122,silver


In [69]:
rfm.groupby(["RFM_class"])["RFM_score"].describe()

,count,mean,std,min,25%,50%,75%,max
RFM_class,,,,,,,,
silver,1667.0,5.212358,1.385663,3.0,4.0,5.0,6.0,7.0
gold,1405.0,9.456940,1.113452,8.0,8.0,9.0,10.0,11.0
platinum,1267.0,13.456985,1.126447,12.0,12.0,13.0,14.0,15.0
